In [1]:
import pandas as pd
import requests
from datetime import date,datetime, timedelta
import numpy as np

In [2]:
def get_dataframe(api_url, data='', id_municipio=''):
    '''Método básico para converter uma chamada a API em um objeto Dataframe'''
    
    api_url = api_url + '?'
    if(id_municipio != ''): 
        api_url = '{}idMunicipio={}&'.format(api_url, id_municipio)
    
    if(data != ''): 
        api_url = '{}data={}&'.format(api_url, data)
    
    result = requests.get(api_url)
    result = result.json()    
    result = pd.DataFrame.from_dict(result)
    
    if(id_municipio != ''): 
        result.insert(0, 'idMunicipio', id_municipio)
        
    return result

In [3]:
api_endpoint = "https://indicadores.integrasus.saude.ce.gov.br/api/coronavirus/qtd-por-dia-tipo"
municipios = get_dataframe('https://indicadores.integrasus.saude.ce.gov.br/api/municipio')

qtd_por_dia_municipio = [get_dataframe(api_endpoint, id_municipio=idm) for idm in municipios.id]

[   idMunicipio      tipo  quantidade        data
 0       230010  Suspeito           1  02/04/2020
 1       230010  Suspeito           1  07/04/2020,
    idMunicipio      tipo  quantidade        data
 0       230015  Suspeito           1  22/03/2020
 1       230015  Suspeito           1  23/03/2020
 2       230015  Suspeito           1  28/03/2020
 3       230015  Suspeito           1  30/03/2020
 4       230015  Suspeito           2  07/04/2020,
     idMunicipio      tipo  quantidade        data
 0        230020  Suspeito           1  18/03/2020
 1        230020  Suspeito           2  19/03/2020
 2        230020  Suspeito           1  20/03/2020
 3        230020  Suspeito           1  23/03/2020
 4        230020  Suspeito           3  24/03/2020
 5        230020  Suspeito           5  27/03/2020
 6        230020  Suspeito           1  28/03/2020
 7        230020  Suspeito           2  29/03/2020
 8        230020  Suspeito           3  30/03/2020
 9        230020  Suspeito           3

In [4]:
qtd_por_dia = pd.concat(qtd_por_dia_municipio, axis=0, ignore_index=True, sort=True)
qtd_por_dia['data'] = pd.to_datetime(qtd_por_dia['data'], format='%d/%m/%Y')
qtd_por_dia

,data,idMunicipio,quantidade,tipo
0,2020-04-02,230010,1.0,Suspeito
1,2020-04-07,230010,1.0,Suspeito
2,2020-03-22,230015,1.0,Suspeito
3,2020-03-23,230015,1.0,Suspeito
4,2020-03-28,230015,1.0,Suspeito
...,...,...,...,...
1464,2020-04-07,231410,1.0,Suspeito
1465,2020-04-09,231410,1.0,Suspeito
1466,NaT,231410,1.0,Suspeito
1467,2020-03-30,231300,1.0,Suspeito


Filtering by the [notified start date](https://g1.globo.com/ce/ceara/noticia/2020/03/15/tres-primeiros-casos-de-coronavirus-no-ceara-sao-confirmados-pela-secretaria-da-saude.ghtml), since there seems to be some values in the API from before the actual beggining 

In [5]:
qtd_por_dia.dropna(inplace=True)
inicio = datetime(year=2020,month=3, day=14)
fim    = datetime.today()
qtd_por_dia = qtd_por_dia[(qtd_por_dia['data'] >= inicio) & (qtd_por_dia['data'] <= fim)]
qtd_por_dia

,data,idMunicipio,quantidade,tipo
0,2020-04-02,230010,1.0,Suspeito
1,2020-04-07,230010,1.0,Suspeito
2,2020-03-22,230015,1.0,Suspeito
3,2020-03-23,230015,1.0,Suspeito
4,2020-03-28,230015,1.0,Suspeito
...,...,...,...,...
1463,2020-04-06,231410,4.0,Suspeito
1464,2020-04-07,231410,1.0,Suspeito
1465,2020-04-09,231410,1.0,Suspeito
1467,2020-03-30,231300,1.0,Suspeito


In [6]:
qtd_por_dia['tipo'].value_counts()

Suspeito      1231
Confirmado     149
Óbito           35
Name: tipo, dtype: int64

In [7]:
qtd_nosuspect = qtd_por_dia.query(" tipo == ['Confirmado', 'Óbito'] ")
qtd_nosuspect

,data,idMunicipio,quantidade,tipo
49,2020-04-06,230075,1.0,Confirmado
74,2020-03-16,230100,1.0,Confirmado
75,2020-03-21,230100,5.0,Confirmado
76,2020-03-23,230100,2.0,Confirmado
77,2020-03-26,230100,2.0,Confirmado
...,...,...,...,...
1416,2020-03-27,231340,1.0,Óbito
1417,2020-03-27,231340,1.0,Confirmado
1418,2020-04-10,231340,1.0,Confirmado
1419,2020-04-11,231340,1.0,Confirmado


In [32]:
no_cities = qtd_nosuspect.drop('idMunicipio', 1)

In [33]:
no_cities = no_cities.pivot_table(index='data', columns='tipo', aggfunc=np.sum)
no_cities

quantidade      
tipo       Confirmado Óbito
data                       
2020-03-16        4.0   NaN
2020-03-17       19.0   NaN
2020-03-18        4.0   NaN
2020-03-19       28.0   NaN
2020-03-20       28.0   NaN
2020-03-21       33.0   NaN
2020-03-22       36.0   NaN
2020-03-23       30.0   NaN
2020-03-24       38.0   2.0
2020-03-25       31.0   1.0
2020-03-26       59.0   2.0
2020-03-27       41.0   3.0
2020-03-28       25.0   2.0
2020-03-29       34.0   3.0
2020-03-30       21.0   3.0
2020-03-31       32.0   2.0
2020-04-01       62.0   2.0
2020-04-02      134.0   7.0
2020-04-03      100.0   7.0
2020-04-04       73.0   7.0
2020-04-05      175.0   4.0
2020-04-06      113.0   9.0
2020-04-07      152.0   4.0
2020-04-08      123.0   5.0
2020-04-09       94.0   4.0
2020-04-10       86.0   6.0
2020-04-11       71.0   1.0

In [34]:
no_cities.columns = no_cities.columns.get_level_values(1)
no_cities.reset_index(level='data', col_level=1)

tipo,data,Confirmado,Óbito
0,2020-03-16,4.0,NaN
1,2020-03-17,19.0,NaN
2,2020-03-18,4.0,NaN
3,2020-03-19,28.0,NaN
4,2020-03-20,28.0,NaN
5,2020-03-21,33.0,NaN
6,2020-03-22,36.0,NaN
7,2020-03-23,30.0,NaN
8,2020-03-24,38.0,2.0
9,2020-03-25,31.0,1.0


In [56]:
no_cities['ntotal_Conf'] = no_cities['Confirmado'].cumsum()
no_cities['ntotal_Ob'] = no_cities['Óbito'].cumsum()
no_cities

tipo,Confirmado,Óbito,ntotal_Conf,ntotal_Ob
data,,,,
2020-03-16,4.0,NaN,4.0,NaN
2020-03-17,19.0,NaN,23.0,NaN
2020-03-18,4.0,NaN,27.0,NaN
2020-03-19,28.0,NaN,55.0,NaN
2020-03-20,28.0,NaN,83.0,NaN
2020-03-21,33.0,NaN,116.0,NaN
2020-03-22,36.0,NaN,152.0,NaN
2020-03-23,30.0,NaN,182.0,NaN
2020-03-24,38.0,2.0,220.0,2.0


In [58]:
no_cities.to_csv('CE-cases_by_date.csv')